# First look at computing derivatives analytically
---
In this notebook we try to test the features of sympy automatic derivative calculator. Given an energy function, we want to compute the following:
- The force
$$\vec{F} = - \nabla E$$
- Force jacobians for implicit euler integration:
$$\frac{\partial \vec{F}}{\partial \vec{x}}\quad\quad\frac{\partial \vec{F}}{\partial \vec{v}}$$
- The derivative of the force wrt the phisical parameters (for differenciable simulations)
$$\frac{\partial \vec{F}}{\partial \vec{p}}$$

In [182]:
import sympy as sym

x1, x2, y1, y2, z1, z2 = sym.symbols("x1 x2 y1 y2 z1 z2")
x1, x2, y1, y2, z1, z2 = sym.symbols("x1 x2 y1 y2 z1 z2")
p1 = sym.Matrix([x1, y1, z1])
p2 = sym.Matrix([x2, y2, z2])
L2 = (p1 - p2).dot(p1-p2)
L = sym.Symbol("L", positive=True)

energy = sym.Rational(1,2) * sym.Symbol("k") * ( L - sym.Symbol("L0"))**2
denergy = energy
energy = energy.subs(L, sym.sqrt(L2))
denergy

k*(L - L0)**2/2

In [211]:
force1 = [sym.diff(energy, sym.Symbol(l)) for l in ["x1", "y1", "z1"]]
force2 = [sym.diff(energy, sym.Symbol(l)) for l in ["x2", "y2", "z2"]]

vforce1 = sym.Matrix(force1)
vforce1 = sym.simplify(vforce1.subs(L2, L**2))
vforce1

Matrix([
[k*(L - L0)*(x1 - x2)/L],
[k*(L - L0)*(y1 - y2)/L],
[k*(L - L0)*(z1 - z2)/L]])

In [212]:
dfdx1 = []
for f1 in force1:
    dfdx1.append([])
    for l1 in ["x1", "y1", "z1"]:
        dfdx1[-1].append(sym.diff(f1, l1))

In [213]:
test = sym.simplify(sym.Matrix(dfdx1).subs(L2, L**2))
test

Matrix([
[k - L0*k/L + L0*k*(x1 - x2)**2/L**3,       L0*k*(x1 - x2)*(y1 - y2)/L**3,       L0*k*(x1 - x2)*(z1 - z2)/L**3],
[      L0*k*(x1 - x2)*(y1 - y2)/L**3, k - L0*k/L + L0*k*(y1 - y2)**2/L**3,       L0*k*(y1 - y2)*(z1 - z2)/L**3],
[      L0*k*(x1 - x2)*(z1 - z2)/L**3,       L0*k*(y1 - y2)*(z1 - z2)/L**3, k - L0*k/L + L0*k*(z1 - z2)**2/L**3]])

In [214]:
from sympy.codegen.cutils import render_as_source_file
render_as_source_file(test.tolist())

'\n\n{{k - L0*k/L + L0*k*pow(x1 - x2, 2)/pow(L, 3), L0*k*(x1 - x2)*(y1 - y2)/pow(L, 3), L0*k*(x1 - x2)*(z1 - z2)/pow(L, 3)}, {L0*k*(x1 - x2)*(y1 - y2)/pow(L, 3), k - L0*k/L + L0*k*pow(y1 - y2, 2)/pow(L, 3), L0*k*(y1 - y2)*(z1 - z2)/pow(L, 3)}, {L0*k*(x1 - x2)*(z1 - z2)/pow(L, 3), L0*k*(y1 - y2)*(z1 - z2)/pow(L, 3), k - L0*k/L + L0*k*pow(z1 - z2, 2)/pow(L, 3)}}'

In [215]:
dfdp = []
for f1 in force1:
    dfdp.append([])
    for l1 in ["k", "L0"]:
        dfdp[-1].append(sym.diff(f1, l1))

In [216]:
test2 = sym.simplify(sym.Matrix(dfdp).subs(L2, L**2))
test2

Matrix([
[(L - L0)*(x1 - x2)/L, k*(-x1 + x2)/L],
[(L - L0)*(y1 - y2)/L, k*(-y1 + y2)/L],
[(L - L0)*(z1 - z2)/L, k*(-z1 + z2)/L]])

In [217]:
render_as_source_file(test2.tolist())

'\n\n{{(L - L0)*(x1 - x2)/L, k*(-x1 + x2)/L}, {(L - L0)*(y1 - y2)/L, k*(-y1 + y2)/L}, {(L - L0)*(z1 - z2)/L, k*(-z1 + z2)/L}}'

In [219]:
keyword = "@PARAMETER"
substitute = render_as_source_file(test.tolist())
with open("test.cpp", "r") as sourcefile:
    read_content = sourcefile.read()
    iind = read_content.find(keyword)
    find = iind + len(keyword)
    new_content = read_content[0:iind] + substitute + read_content[find:-1]
    generatedfile = open("output.cpp", "w")
    generatedfile.write(new_content)
    generatedfile.close()